# 11 物理中重要的数据结构

包括：
1.散列表：用于在数据之间创建关联关系。
2.数据框：与数组或表格类似，但添加了一些新功能，能够更好地用于试验数据。
3.B树：用来管理数组块和层次。
4.K-d树：分割多维空间的数据结构，可用来分析在空间中的最近邻点。

## 11.1 散列表

散列表是一种特殊的类型，将键（key）和值（value）对应起来，其中键必须是唯一的。

#### 与表的关系

相当于一个表中，键为其中一列，而值为另一列。
表会根据键的值对表长取模的结果进行排序。
表中可以有空行或缺失行。
表中在插入和查找时都根据键的散列值确定，因此必须使用一个非常好的机制来对键生成散列值。

表格中数据的行索引i与表达式hash（key）％table_len的结果完全相同。
重新计算索引表达式就能跳转到表中的对应地方，以此来快速访问基于键的元素。

#### 散列表操作的复杂度

散列表在关联数据方面的主要创新是在寻找值时无需测试所有键的等同性。
即使是更加复杂的查找机制，其平均查找速度也比搜索一个等价的数据结构（如元组列表）要快得多。

对于获取、设置、删除项，散列表的平均复杂度为1。
对大小差异的同结构散列表，，检索项目的时间完全相同。
最坏的情况下，获取、设置、删除项目的开销最多为O(N)，其中N是表的大小。

### 11.1.1 调整大小

**散列表空间不足以存储新项目时就必须调整大小。**

通常需要在内存中分配新空间，复制所有数据并重新组织每项在数组中的位置。
重排是因为散列表中行索引是对表的长度取模得到。

散列表会自动控制表的大小和布局，用户只需要担心调整大小的开销。
与一次性插入多个值相比，分次依次插入的开销更昂贵。

不同的散列表实现用不同的策略来决定何时调整大小（一半、四分之三、全部、从不调整）；
以及调整多少（翻倍、增加一半、不增加）。
调整大小可以解决散列表空间不足的问题。

### 11.1.2 冲突

**新键的散列值与表中现有键的散列值相同时，就会发生*散列冲突*。**

对于基本数据类型，值的散列值全相同（都是0）：
<code>hash('') == hash(0) == hash(0.0) == hash(False) == 0</code>

**即使对于随机的键值，也存在散列冲突的问题。**
两个键的散列值相同的可能性大于两个键的散列值都是指定散列值的概率（生日悖论）。

变量s为表的大小，N是某个散列函数中生成的不同散列值的数量。散列冲突的概率$p_c(s)$：
$$p_c(s)=1-e^{\frac{-s(s-1)}{2N}}$$
进一步近似：
$$p_c(s)=\frac{s^2}{2N}$$

对于Python字典，N为$2^{64}$。在大约十亿个项之后，散列冲突的概率开始显著增加，超过100亿项就总会发生散列冲突。空间的总容量约为$1.844\times 10^{19}$项

#### 处理冲突时广泛遵循的策略

1.每个索引都是一个存储桶（bucket）或键/值对的列表。根据散列值先找到相应的桶，然后对桶进行线性搜索来找到正确的值。用线性搜索和增加数据结构的复杂度为代价来降低散列表调整次数。这种方法称为拉链法。
2.在发生碰撞时，以可预测和可逆的方式改变数据的散列值。每次发生碰撞都会改变散列值。查找时首先尝试原散列值所在的位置，然后是修改之后的值。这称为开放寻址法，Python字典在实现时采用了这种策略。此时表中的所有项都是平直排列，无需搜索子列表。但代价是无法从键的散列值直接得到键的索引。散列表完整的历史记录（数据项插入和删除的时间点）决定了索引值。
3.始终调整到零碰撞点。这种方式适用于所有键都是相同的类型。有时被称为双向或双射散列，因为每个键的散列值是唯一的，散列就如同由键值唯一确定。

## 11.2 数据框

数据框（Data frame）：表和结构数组的抽象。
数据框能够优雅地处理丢失的数据。
R语言中原生实现了数据框，大部分情况下数据框用于统计处理。
Python可用pandas包实现数据框和相关的工具。

数据框是由名为series的命名列组成的高效表格。series可以动态地从数据框中添加或删除。
一个series可以认为是一个一维NumPy数组（含有dtype）的值以及一个相应的索引数组。
索引用来定位series的值。如果未提供索引，则索引为从0到到series长度减1的整数。
索引可以是字符串。

**以下是基于Pandas的实现。**

### 11.2.1 Series

#### Series基础

Pandas使用Series类。
Series类实际上是一个具有可选关联索引的一维NumPy数组，可以认为Series对象使用了大量NumPy风格的操作。

#### Series创建

Series可以使用类似数组的机制创建，共享NumPy数组使用的原始类型系统。
打印Series，左侧的值是索引。
索引可以使用index参数创建。

索引本身非常重要，它决定了如何访问Series的元素。
是一个不可变的ndarray，只由可散列的数据类型组成，散列性确保索引的元素可安全地从Series中检索值。
索引用法也和字典类似。

可以从字典中创建Series。
键是索引，并且根据键对元素进行排序。

#### Series运算

可以对Series或多个Series执行算术和其他操作。
个
Series交互时，如果一个Series中存在具有特定索引的元素，另一个Series中没有这个索引，则结果中缺失索引的值为NaN。
这意味着数据集只会增长，并且不会丢失索引。
缺点：在数据中心出现NaN可能并不是期望的行为。可能会影响数据计数。
优点：可以非常清楚地看出操作的输入Series没有共同基础。

处理NaN的方法：
1.确保原来的Series都共享一个共同的索引。
2.在其他操作完成之后过滤或屏蔽NaN元素。


In [ ]:
# 基本应用
import pandas as pd

# 创建一个Series
s0 = pd.Series([42, 43, 44], dtype='f8')  # 64位浮点数类型的Series
print("s0为：\n", s0)
# 创建带有索引的Series
s = pd.Series([42, 43, 44],
               index=["electron",
                      "proton",
                      "neutron"])  # index参数为索引
print("s为：\n", s)
# 索引使用
print("s['electron']: ", s['electron'])  # 单值引用
print("s['electron':'proton']: ", s['electron':'proton'])  # 切片
print("s[1:]", s[1:])  # 开始索引

In [ ]:
# 字典化应用
t = pd.Series({'electron': 6,
               'neutron': 28,
               'proton': 496,
               'neutrino': 8128})
print("t为：\n", t)
# 计算s2+t
print("s+t为：\n", s+t)

### 11.2.2 DataFrame结构

数据框（DataFrame）对象可以理解为Series的集合。
这些Series不需要使用相同的索引，但建议索引相同。
*索引就像希尔伯特空间的基，同一组基能够更好的进行运算。*

数据框是类似于NumPy结构化数组或PyTables表的类表格结构。
提供了其他表数据结构不一定具有的许多有用的语义。
与其他类似表结构不同：其中的列是Series而不是数组。

（1）从数组、列表、Series的字典中创建数据框。
字典的键做为列名，使用pd.DataFrame创建数据框。
（2）以从NumPy结构化数组或元组列表中创建数据框。
数据框可以从CSV文件、HDF5文件（通过PyTables）、HTML表、SQL和各种其他格式保存和加载。

数据框可以切片或添加或删除行。但数据框也有与Series相同的复杂的索引语义。

数据框的一个主要创新是能够方便地添加和删除列。
使用NumPy结构化数组向现有数组添加新列需要创建一个新的组合类型的dtype来表示新表，将新列数据与现有表交叉，并将所有数据复制到新的结构化数组中。
数据框中的列等于索引。
列访问和操作通过类似字典的索引进行。


In [ ]:
# DataFrame的创建
df = pd.DataFrame({'S': s, 'T': t})
print("创建的数据框为：\n", df)
# DataFrame操作
print("df[::2]: \n", df[::2])  # 每隔两个元素进行切片
df['M'] = pd.DataFrame({'T': [-8128]}, index=s.index)  # 利用赋值添加列
print("修改后的df为：\n", df)
print("转置df：\n", df.T)  # 使用T属性转置（这是转置符号？）
print("按照是否小于42来创建布尔掩模数据框：\n", df < 42)  # 实际上创建了另一个数据框

In [ ]:
# 数据框的列操作
print("df['T']: \n", df['T'])  # 列名作索引
df['small'] = df['T'] < 100  # 添加操作
print("修改后的df: \n", df)
del df['small']  # 删除small列
print("删除small列之后: \n", df)

## 11.3 B树

B树是用于搜索大块数据时最常见的数据结构之一，适用于数据库。
HDF5本身就是基于B树。数据集中的块通过B树存储和管理。
层次结构本身（多个数组和表的集合）也通过B树来管理。

B树是一种所有节点都以**广度优先**的方式排序的树数据结构。
每个节点可以具有多个子节点。
每个节点可以存储多个值，且每个节点中值的数量也可以不同。
单值节点退化成二叉树。

B树（以及二叉搜索树）可以旋转。这意味着可以在不破坏搜索和排序属性的情况下重新排列节点来转换结构。

```
      [5]              [2]
     /   \            /   \
   [2]   [8]  -->   [1]   [5]
  /   \                  /   \
[1]   [3]              [3]   [8]
```

B树非常适合以非线性的方式组织数组数据。
1.数组的索引确定数组在树中的节点位置。
2.树本身管理所有节点的位置。
3.节点管理所分配的数据块。
节点能够在任意索引处插入和删除数组，因此可以表示含有丢失的块、无限长并且可扩展的数组。

B树提高性能并简化逻辑的规则：
1.树的高度h是常数。所有叶节点（终端节点）存在于相同的高度。
2.根节点的高度为0。
3.所有节点中子节点的最大数目m小于一个常数。
4.为了平衡，节点应该尽可能在树上均匀分布。

树的节点数量（n）决定了树的大小。搜索复杂度：O($\log_2n$)。

每当需要在大且可能稀疏的数据集内快速查找元素时，建议使用B树。
实现Python库：
1.btree：类似于字典的C扩展模块。
2.BTrees：为Zope Object Database（ZODB）优化的通用B树实现。
3.blist：以B+树（一种B树变体）实现了类似列表、元组、集合、字典的数据结构。

## 11.4 k-d树

k-d树（或k维树）适合在k维空间中找到某点的最近邻点。
表示场景为坐标系中的点，按树形排序。
【使用场景举例】求解几何偏微分方程时，当前单元的主要影响来自周围单元。将几何问题分解为k-d树可以更快地找到最近的相邻单元。
核心思想：任何点（连同问题边界）定义了一个(k-1)维超平面，将剩余空间分成两个部分。

k-d树中的点可以放置到类似于二叉搜索树的结构中。
排序是基于沿着轴a的点，并且a等于点的深度对维数k的模数。

k-d树被实例化后一般就不能修改。通常有get或query方法，但没有插入或删除方法。
重构空间需要创建一个全新的树。
空间复杂度：O($log_kn$)
k很小时最有效。



In [1]:
# k-d树实现，k=2
class Node(object):  # 含有节点的树。
    # 节点由二维点定义。由于这是一个二叉搜索树，因此左右各有一个节点。
    def __init__(self, point, left=None, right=None):
        self.point = point
        self.left = left
        self.right = right

    # 用字符串表示节点在树中的相对位置。
    def __repr__(self):
        isleaf = self.left is None and self.right is None
        s = repr(self.point)
        if not isleaf:
            s = "[" + s + ":"
        if self.left is not None:
            s += "\n left = " + "\n ".join(repr(self.left).split('\n'))
        if self.right is not None:
            s += "\n right = " + "\n ".join(repr(self.right).split('\n'))
        if not isleaf:
            s += "\n ]"
        return s


# 递归函数，返回给定二维点列表的根节点。该函数会自动生成平衡的K-d树。
def kdtree(points, depth=0):
    if len(points) == 0:
        return None
    k = len(points[0])
    a = depth % k
    points = sorted(points, key=lambda x: x[a])
    i = int(len(points) / 2) # middle index, rounded down
    node_left = kdtree(points[:i], depth + 1)
    node_right = kdtree(points[i+1:], depth + 1)
    node = Node(points[i], node_left, node_right)
    return node

points = [(1, 2), (3, 2),
          (5, 5), (2, 1),
          (4, 3), (1, 5)]
root = kdtree(points)
print(root)


[(3, 2):
 left = [(1, 2):
  left = (2, 1)
  right = (1, 5)
  ]
 right = [(5, 5):
  left = (4, 3)
  ]
 ]


#### KDTree类

scipy.spatial中的KDTree类。
基于NumPy，不仅能生成，还有查找等功能。
搜索空间超过userdefinable参数时，KDTree会转到暴力搜索。

data属性：示点的NumPy数组。
如果点最初是一个NumPy数组，那么data属性是一个视图，而不是副本。

query()方法可在空间中的任意位置处获取点序列，并返回N个最近点的信息，其中包括这些点的距离数组以及点本身的数据数组的索引。
query()不返回点所在的单元格。
在物理系统中比较有用，可用于场的计算，包括标量场和向量场。


In [3]:
from scipy.spatial import KDTree


points = [(1, 2), (3, 2),
          (5, 5), (2, 1),
          (4, 3), (1, 5)]
tree = KDTree(points)
print(tree.data)  # 输出data属性数组

# query()获取最近的点
dist, idx = tree.query([(4.5, 1.25)])  # dist为距离，idx为索引
print(dist, "\n", idx)  # 不返回具体的点
print(tree.data[idx])  # 通过索引获得点的位置

[[1. 2.]
 [3. 2.]
 [5. 5.]
 [2. 1.]
 [4. 3.]
 [1. 5.]]
[1.67705098] 
 [1]
[[3. 2.]]
